# How to use Python to create beautiful plots for scientific publications ?
# 1. Plotting data on a map
### [Basemap](https://matplotlib.org/basemap/index.html) 
### [Install Basemap](https://github.com/matplotlib/basemap)
### [Cartopy](https://scitools.org.uk/cartopy/docs/v0.15/gallery.html)
### [GeoCAT](https://www.ncl.ucar.edu/Document/Pivot_to_Python/september_2019_update.shtml)

##### %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 
# Import Packages

In [ ]:
import cmaps
import cmocean 
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap

# Load data

In [ ]:
# Load grid information
grid_file = Dataset('/Users/z3533156/OneDrive - UNSW/Python/PUG/EACouter_varres_grd_mergedBLbry_uhroms.nc', 'r')
lon_roms  = grid_file.variables['lon_rho'][:,:]
lat_roms  = grid_file.variables['lat_rho'][:,:]
grid_file.close()
# Load climatology
clim_file = Dataset('/Users/z3533156/OneDrive - UNSW/Python/PUG/tide_annual_mean.nc', 'r')
clim_v    = clim_file.variables['v_northward'][0,29,:,:]
clim_v[np.where(lon_roms>156)]=np.nan
clim_file.close()
# Load composite EKE, KmKe, PeKe, SLA, SSH, V during the high-EKE periods
dataset     = sio.loadmat('/Users/z3533156/OneDrive - UNSW/Python/PUG/Figure2_data.mat')
EKE_high    = dataset['EKE_high']-dataset['EKE_clim']
EKE_high_h  = dataset['EKE_high_pvalue']
KmKe_high   = dataset['KmKe_high']*1e4
KmKe_high_h = dataset['KmKe_high_pvalue']
PeKe_high   = dataset['PeKe_high']*1e4
PeKe_high_h = dataset['PeKe_high_pvalue']
SLA_high    = dataset['SLA_high']
SSH_high    = dataset['SSH_high']
ugos_high   = dataset['ugos_high']
vgos_high   = dataset['vgos_high']
ugosa_high  = dataset['ugosa_high']
vgosa_high  = dataset['vgosa_high']
SLA_high_h  = dataset['SLA_high_pvalue']
vgos_high_h = dataset['vgos_high_pvalue']
# Load composite EKE, KmKe, PeKe, SLA, SSH, V during the low-EKE periods
EKE_low     = dataset['EKE_low']-dataset['EKE_clim']
EKE_low_h   = dataset['EKE_low_pvalue']
KmKe_low    = dataset['KmKe_low']*1e4
KmKe_low_h  = dataset['KmKe_low_pvalue']
PeKe_low    = dataset['PeKe_low']*1e4
PeKe_low_h  = dataset['PeKe_low_pvalue']
SLA_low     = dataset['SLA_low']
SSH_low     = dataset['SSH_low']
ugos_low    = dataset['ugos_low']
vgos_low    = dataset['vgos_low']
ugosa_low   = dataset['ugosa_low']
vgosa_low   = dataset['vgosa_low']
SLA_low_h   = dataset['SLA_low_pvalue']
vgos_low_h  = dataset['vgos_low_pvalue']

## Colormaps
### [Colormaps in Matplotlib](https://matplotlib.org/stable/tutorials/colors/colormaps.html)
### [cmocean](https://matplotlib.org/cmocean/)
### [cmaps using NCL colorbars](https://github.com/hhuangwx/cmaps)
## Layout of multipanels

In [ ]:
# Plot composite SLA and SSH
fig = plt.figure(figsize=(50, 50))
labelfont = 90
padspacescale = 50
labelpadscale = 20
linefont = 12
# Create a basemap of the EAC region
m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
m.fillcontinents(color='0.95', lake_color='white')
m.drawstates(linewidth=0.7*linefont, linestyle='dashed', color='k', antialiased=1, ax=None, zorder=None)
# plot the boundary of the EAC-ROMS domain
x, y   = m(lon_roms, lat_roms)
m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='k')
m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='k')
m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
# Add the Tasman EKE Box
lon   = np.array([lon_roms[84,114],lon_roms[149,114],lon_roms[149,184],lon_roms[84,184],lon_roms[84,114]])
lat   = np.array([lat_roms[84,114],lat_roms[149,114],lat_roms[149,184],lat_roms[84,184],lat_roms[84,114]])
x1,y1 = m(lon,lat)
m.plot(x1,y1,linewidth=linefont,linestyle='solid', color='black')
# 1. Plot composite SLA
cmaps1  = cmocean.cm.curl
cmaps2  = cmaps.BlueWhiteOrangeRed
levels1 = np.linspace(-0.2,0.2,100)
CB1     = m.contourf(x, y, SLA_high, cmap=cmaps2,levels=levels1,origin='lower',extend='both') 
# 2. Add stippling
m_scale = 1000
m_colors1 = 'gray'
m_colors2 = 'tab:green'
yy = np.arange(10, y.shape[0], 10)
xx = np.arange(10, x.shape[1], 10)
points = np.meshgrid(yy, xx)
point_index = tuple(points)
point_x  = x[point_index]
point_y  = y[point_index]
point_z  = SLA_high_h[point_index]
point_x  = np.ravel(point_x)
point_y  = np.ravel(point_y)
point_z  = np.ravel(point_z)
point_xp = point_x[~np.isnan(point_z)]
point_yp = point_y[~np.isnan(point_z)]
CB2 = m.scatter(point_xp,point_yp,s=m_scale,c=m_colors2,marker='.')
# 3. Add composite SSH
CS1 = m.contour(x, y, SSH_high,levels=np.linspace(0,0.8,17),linewidths=0.4*linefont,colors='black',linestyles='dashed')
# Highlight 0.03 m SSH contour
CS2 = m.contour(x, y, SSH_high,np.linspace(0.3,0.3,1),linewidths=linefont,colors='black',linestyles='solid')
# 4. Add the velocity anomalies vectors
ua = ugosa_high[point_index]
va = vgosa_high[point_index]
y_quiver = 0.9
x_quiver = 0.10
cur_vara = 0.5
str_vara = ['0.5 m/s']
Qa  = m.quiver(point_x,point_y,ua,va,color='black',width=0.004,scale=3.0)
qka = plt.quiverkey(Qa, x_quiver, y_quiver, cur_vara, str_vara[0],
                    labelpos='N',color='black',linewidth=linefont*1.0,fontproperties={'size': labelfont})
# Add colorbar
tick_marks = np.linspace(-0.2,0.2,5)
cbaxes = fig.add_axes([0.22, 0.08, 0.58, 0.02])
cb = plt.colorbar(CB1,orientation='horizontal',cax = cbaxes)
cb.set_label('SLA (m)', fontsize=labelfont,rotation=0,labelpad=labelpadscale)
cb.set_ticks(tick_marks)
cb.ax.tick_params(labelsize=labelfont)

In [ ]:
# Plot composite SLA and SSH
fig = plt.figure(figsize=(50, 50))
labelfont = 90
padspacescale = 50
labelpadscale = 20
linefont = 12
# Create a basemap of the EAC region
m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
            llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
m.fillcontinents(color='0.95', lake_color='white')
m.drawstates(linewidth=0.7*linefont, linestyle='dashed', color='k', antialiased=1, ax=None, zorder=None)
# plot the boundary of the EAC-ROMS domain
x, y   = m(lon_roms, lat_roms)
m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='k')
m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='k')
m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
# Add the Tasman EKE Box
lon   = np.array([lon_roms[84,114],lon_roms[149,114],lon_roms[149,184],lon_roms[84,184],lon_roms[84,114]])
lat   = np.array([lat_roms[84,114],lat_roms[149,114],lat_roms[149,184],lat_roms[84,184],lat_roms[84,114]])
x1,y1 = m(lon,lat)
m.plot(x1,y1,linewidth=linefont,linestyle='solid', color='black')
# 1. Plot composite SLA
cmaps1  = cmocean.cm.curl
cmaps2  = cmaps.BlueWhiteOrangeRed
levels1 = np.linspace(-0.2,0.2,100)
CB1     = m.contourf(x, y, SLA_high, cmap=cmaps2,levels=levels1,origin='lower',extend='both')  
# 2. Add stippling
m_scale = 1000
m_colors1 = 'gray'
m_colors2 = 'tab:green'
yy = np.arange(10, y.shape[0], 10)
xx = np.arange(10, x.shape[1], 10)
points = np.meshgrid(yy, xx)
point_index = tuple(points)
point_x  = x[point_index]
point_y  = y[point_index]
point_z  = SLA_high_h[point_index]
point_x  = np.ravel(point_x)
point_y  = np.ravel(point_y)
point_z  = np.ravel(point_z)
point_xp = point_x[~np.isnan(point_z)]
point_yp = point_y[~np.isnan(point_z)]
CB2 = m.scatter(point_xp,point_yp,s=m_scale,c=m_colors2,marker='.')
# 3. Add composite SSH
CS1 = m.contour(x, y, SSH_high,levels=np.linspace(0,0.8,17),linewidths=0.4*linefont,colors='black',linestyles='dashed')
# Highlight 0.03 m SSH contour
CS2 = m.contour(x, y, SSH_high,np.linspace(0.3,0.3,1),linewidths=linefont,colors='black',linestyles='solid')
# 4. Add the velocity anomalies vectors
ua = ugosa_high[point_index]
va = vgosa_high[point_index]
y_quiver = 0.9
x_quiver = 0.10
cur_vara = 0.5
str_vara = ['0.5 m/s']
Qa  = m.quiver(point_x,point_y,ua,va,color='black',width=0.004,scale=3.0)
qka = plt.quiverkey(Qa, x_quiver, y_quiver, cur_vara, str_vara[0],
                    labelpos='N',color='black',linewidth=linefont*1.0,fontproperties={'size': labelfont})
# Add colorbar
tick_marks = np.linspace(-0.2,0.2,5)
cbaxes = fig.add_axes([0.22, 0.08, 0.58, 0.02])
cb = plt.colorbar(CB1,orientation='horizontal',cax = cbaxes)
cb.set_label('SLA (m)', fontsize=labelfont,rotation=0,labelpad=labelpadscale)
cb.set_ticks(tick_marks)
cb.ax.tick_params(labelsize=labelfont)

# Incorporate multipanels into one figure

![Jupyter](./Figure1.png)

In [ ]:
# EKE Box locations
lon   = np.array([lon_roms[84,114],lon_roms[149,114],lon_roms[149,184],lon_roms[84,184],lon_roms[84,114]])
lat   = np.array([lat_roms[84,114],lat_roms[149,114],lat_roms[149,184],lat_roms[84,184],lat_roms[84,114]])
x1,y1 = m(lon,lat)
# SLA Box locations
lon   = np.array([lon_roms[239,79],lon_roms[309,79],lon_roms[309,159],lon_roms[239,159],lon_roms[239,79]])
lat   = np.array([lat_roms[239,79],lat_roms[309,79],lat_roms[309,159],lat_roms[239,159],lat_roms[239,79]])
x2,y2 = m(lon,lat)
# Stippling and vectors locations
m     = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
              llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
x, y  = m(lon_roms, lat_roms)
yy = np.arange(10, y.shape[0], 10)
xx = np.arange(10, x.shape[1], 10)
points = np.meshgrid(yy, xx)
point_index = tuple(points)
point_x  = np.ravel(x[point_index])
point_y  = np.ravel(y[point_index])
x_quiver = 0.18
y_quiver = 0.9
cur_var  = 0.5
cur_vara = 0.3
str_var  = ['0.5m/s']
scale    = 1.3
m_scale  = 1000
m_colors1 = 'gray'
m_colors2 = 'tab:green'
# Contour and colorbar scales
levels1 = np.linspace(-0.2,0.2,100)
levels2 = np.linspace(-2.5,2.5,100)
levels4 = np.linspace(-0.2,0.2,100)
levels5 = np.linspace(-0.5,0.5,100)
tick_marks1  = np.linspace(-0.2,0.2,5)
tick_marks2  = np.linspace(-2.5,2.5,6)
tick_marks4  = np.linspace(-0.2,0.2,5)
tick_marks44 = np.linspace(0,0.8,17)
tick_marks5  = np.linspace(-0.5,0.5,5)
cmaps1 = cmocean.cm.curl
cmaps2 = cmaps.BlueWhiteOrangeRed
# Variable for each panel
var_u  = np.array([ugos_high[point_index],ugos_low[point_index]])
var_v  = np.array([vgos_high[point_index],vgos_low[point_index]])
var_composite = np.array([EKE_high,   KmKe_high,   PeKe_high,   SLA_high,   vgos_high,\
                          EKE_low,    KmKe_low,    PeKe_low,    SLA_low,    vgos_low])
var_SSH       = np.array([EKE_high,   KmKe_high,   PeKe_high,   SSH_high,   vgos_high,\
                          EKE_low,    KmKe_low,    PeKe_low,    SSH_low,    vgos_low])
var_p         = np.array([EKE_high_h, KmKe_high_h, PeKe_high_h, SLA_high_h, vgos_high_h,\
                          EKE_low_h,  KmKe_low_h,  PeKe_low_h,  SLA_low_h,  vgos_low_h])
title_labels  = ['(a)EKEA(High)', '(b)KmKe(High)', '(c)PeKe(High)', '(d)SLA(High)', '(e)V(High)',\
                 '(f)EKEA(Low)',  '(g)KmKe(Low)',  '(h)PeKe(Low)',  '(i)SLA(Low)',  '(j)V(Low)']

In [ ]:
# Creat multipanels plot
fig = plt.figure(figsize=(85, 80))
labelfont = 90
padspacescale = 50
labelpadscale = 20
linefont = 12
# Create a plot with 2 rows and 5 columns
gs = gridspec.GridSpec(2,5)
for i in range(10):
    # Get the locations of each panel
    ax = fig.add_subplot(gs[i])
    l, b, w, h = ax.get_position().bounds
    # Create a basemap of the EAC region
    m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat_roms),urcrnrlat= np.nanmax(lat_roms),\
                  llcrnrlon=np.nanmin(lon_roms),urcrnrlon=np.nanmax(lon_roms),resolution='h')
    m.drawparallels(np.arange(-45,-20, 2),labels=[0,0,0,0],linewidth=0.5*linefont,dashes=[2,2],color='.5',fontsize=labelfont)
    m.drawmeridians(np.arange(145,165, 5),labels=[0,0,0,0],linewidth=0.5*linefont,dashes=[2,2],color='.5',fontsize=labelfont)
    m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
    m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
    m.fillcontinents(color='0.95', lake_color='white')
    m.drawstates(linewidth=0.7*linefont, linestyle='dashed', color='k', antialiased=1, ax=None, zorder=None) 
    # set location for each row
    if i<5:
        b=b-0.08
        u=var_u[0]
        v=var_v[0]
    else:
        b=b+0.06
        u=var_u[1]
        v=var_v[1]
        # Add longitude in the second row
        m.drawmeridians(np.arange(145,165, 5),labels=[0,0,0,1],linewidth=0.5*linefont,dashes=[2,2],color='.5',fontsize=labelfont)        
    # Add title for each panel
    plt.title(title_labels[i], fontsize=1.1*labelfont,loc='left',pad=padspacescale)   
    # plot the boundary of the EAC-ROMS domain
    m.plot(x[0,:], y[0,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,0], y[:,0],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
    # Add the Tasman EKE Box 
    m.plot(x1,y1,linewidth=linefont,linestyle='solid', color='black')
    # Set length and width for the axis
    ax.spines['bottom'].set_linewidth(linefont)
    ax.spines['left'].set_linewidth(linefont)
    ax.spines['top'].set_linewidth(linefont)
    ax.spines['right'].set_linewidth(linefont)
    plt.tick_params(axis='both',which='major',bottom='on',left='on',length=50.0,width=20,colors='black',direction='out')
    # Stippling locations
    stipple_p=var_p[i]
    point_z=stipple_p[point_index]
    point_z=np.ravel(point_z)
    point_xp=point_x[~np.isnan(point_z)]
    point_yp=point_y[~np.isnan(point_z)]
    # Plot composite data on each panel
    if np.mod(i+1,5)==1:
        # EKE anomalies
        l=l-0.05
        # Add latitude in the first column
        m.drawparallels(np.arange(-45,-20, 2),labels=[1,0,0,0],linewidth=0.5*linefont,dashes=[2,2],color='.5',fontsize=labelfont)        
        CB1  = m.contourf(x, y, var_composite[i], cmap=cmaps1,levels=levels1,origin='lower',extend='both')
        CB11 = m.scatter(point_xp,point_yp,s=m_scale,c=m_colors1,marker='.')
    elif np.mod(i+1,5)==2:
        # KmKe
        l=l-0.034
        CB2  = m.contourf(x, y, var_composite[i], cmap=cmaps1,levels=levels2,origin='lower',extend='both')
        CB22 = m.scatter(point_xp,point_yp,s=m_scale,c=m_colors1,marker='.')
    elif np.mod(i+1,5)==3:
        # PeKe
        l=l-0.018
        CB3  = m.contourf(x, y, var_composite[i], cmap=cmaps1,levels=levels2,origin='lower',extend='both')
    elif np.mod(i+1,5)==4:
        # SSH and SLA
        l=l-0.002
        CB4  = m.contourf(x, y, var_composite[i], cmap=cmaps2,levels=levels4,origin='lower',extend='both')           
        CS1  = m.contour(x, y,  var_SSH[i],levels=tick_marks44,linewidths=0.4*linefont,colors='black',linestyles='dashed')
        CS2  = m.contour(x, y,  var_SSH[i],np.linspace(0.3,0.3,1),linewidths=linefont,colors='black',linestyles='solid')
        # Add SLA Box        
        m.plot(x2,y2,linewidth=linefont,linestyle='solid', color='red')
        CB44 = m.scatter(point_xp,point_yp,s=m_scale,c=m_colors2,marker='.')
    else:   
        # V 
        l=l+0.014
        CB5  = m.contourf(x, y, var_composite[i], cmap=cmaps2,levels=levels5,origin='lower',extend='both')  
        CS   = m.contour(x, y, clim_v,np.linspace(-0.05,0.05,1),linewidths=linefont,colors='gray',linestyles='solid')
        Q    = m.quiver(point_x,point_y,u,v,color='black',width=0.004,scale=5.0)
        qk   = plt.quiverkey(Q, x_quiver, y_quiver, cur_var, str_var[0],labelpos='N',color='black',linewidth=linefont*1.0,fontproperties={'size': labelfont})
    # Set the location for each panel
    ax.set_position([l, b, scale*w, scale*h])
# Add colorbars
cbaxes1     = fig.add_axes([0.08, 0.26, 0.16, 0.01])
cb1 = plt.colorbar(CB1,orientation='horizontal',cax = cbaxes1)
cb1.set_label('EKE ($\mathregular{m^{2}/s^{2}}$)', fontsize=0.7*labelfont,labelpad=0)
cb1.set_ticks(tick_marks1)
cb1.ax.tick_params(labelsize=0.7*labelfont)
cbaxes2     = fig.add_axes([0.26, 0.26, 0.34, 0.01])
cb2 = plt.colorbar(CB2,orientation='horizontal',cax = cbaxes2)
cb2.set_label('($\mathregular{10^{-4}}$ W/$\mathregular{m^{2}}$)', fontsize=0.7*labelfont,labelpad=0)
cb2.set_ticks(tick_marks2)
cb2.ax.tick_params(labelsize=0.7*labelfont)
cbaxes4     = fig.add_axes([0.61, 0.26, 0.16, 0.01])
cb4 = plt.colorbar(CB4,orientation='horizontal',cax = cbaxes4)
cb4.set_label('SLA (m)', fontsize=0.7*labelfont,labelpad=0)
cb4.set_ticks(tick_marks4)
cb4.ax.tick_params(labelsize=0.7*labelfont)
cbaxes5     = fig.add_axes([0.79, 0.26, 0.16, 0.01])
cb5 = plt.colorbar(CB5,orientation='horizontal',cax = cbaxes5)
cb5.set_label('V-component (m/s)', fontsize=0.7*labelfont,labelpad=0)
cb5.set_ticks(tick_marks5)
cb5.ax.tick_params(labelsize=0.7*labelfont)